Кобзарь О.С. Хабибуллин Р.А. 

# Результаты расчета скважин

В данном сводном отчете будут самые полезные графики для визуального анализа

Входными данными являются файлики расчетов типа *Итоговый результат для 1354 время 2019_10_12_17_33_46.csv*
в котором будут либо результаты адаптации, либо результаты восстановления дебитов. DataFrame с пометкой ADAPT - результаты адаптации, DataFrame с пометкой RESTORE или RESTORE2 - результаты восстановления дебитов различными способами. Если метод адатации только один, необходимо активировать флаг *one_restore*

В легенде пометки типа *СУ* обозначают то, что данные со станции управления, *Ш* - шахматки, *модель* - входные или выходные данные из расчетной модели скважины.



In [ ]:
import pandas as pd
import sys
sys.path.append(r"C:\Users\olegk\Documents")
from unifloc.uniflocpy.uTools import plotly_workflow as pw
import datetime
from plotly.offline import download_plotlyjs, plot, iplot, init_notebook_mode
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime
init_notebook_mode(connected=True)

In [ ]:
# TODO подумать, как построить графики для 3 варинтов расчета и больше
# TODO еще для прогноза отдельно добавить
# корреляционные карты нужно сюда?

Нужно указать какая скважина, чтобы не запутаться

In [ ]:
well_name = '1354'
time_mark = datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
one_restore = True
q_esp_nom_m3day = 320
head_esp_nom_m = 1500

adaptation_data = 'Итоговый результат для 1354 время 2019_10_12_22_49_42.csv'
restore_data = 'Итоговый результат для 1354 время 2019_10_12_17_33_46.csv'
#restore_data_2 = 'Итоговый результат для 507 время 2019_10_17_23_17_14.csv'

# Конкретные моменты для данной скважины 1354

In [ ]:
def prepare_columns_in_file(df, mark):
    df.index = pd.to_datetime(df['Unnamed: 0'])
    del df['Unnamed: 0']
    for i in df.columns:
        df = df.rename(columns={i: i + ' (' + mark + ')' })
    return df

def plot_func(df, plot_title_str, plot_in_notebook = True):
    """
    Итоговая функция для построения графиков

    :param data: созданный список из trace
    :param plot_title_str: название графика
    :param filename_str: названия html файлика
    :return: None
    """
    layout = dict(title=plot_title_str, legend_orientation="h")
    data = pw.create_traces_list_for_all_columms(df, chosen_mode='lines+markers', low_memory = False)
    fig = dict(data=data, layout=layout)
    if plot_in_notebook:
        iplot(fig)
    else:
        plot(fig)

Сведем все данные по адаптации и восстановлению в единый DataFrame

In [ ]:

adapt_df = pd.read_csv(adaptation_data)
adapt_df = prepare_columns_in_file(adapt_df, 'ADAPT')
restore_df = pd.read_csv(restore_data)
restore_df = prepare_columns_in_file(restore_df, 'RESTORE')
overall_data = restore_df.join(adapt_df, how = 'outer')

if one_restore == False:
    
    restore_df_2 = pd.read_csv(restore_data_2)
    restore_df_2 = prepare_columns_in_file(restore_df_2,  'RESTORE2')
    overall_data = overall_data.join(restore_df_2, how = 'outer')

In [ ]:
overall_data['Активная мощность (СУ) (ADAPT)'] = overall_data['Активная мощность (СУ) (ADAPT)'] * 1000
overall_data['Загрузка двигателя (СУ) (ADAPT)'] = overall_data['Загрузка двигателя (СУ) (ADAPT)'] / 100

In [ ]:
#for i in (overall_data.columns):
#    print(i)

In [ ]:
#import description_generated.python_api as python_api
#import os
#path_to_addin = os.getcwd()
#path_to_addin = path_to_addin.replace('stuff_to_merge', 'UniflocVBA_7.xlam')
#UniflocVBA = python_api.API(path_to_addin)
#h_m = []
#efficency_perc = []
#power_wt = []
#q_m3day = list(range(1, int(q_esp_nom_m3day * 1.8),10))
#pump_id = UniflocVBA.calc_ESP_id_by_rate(q_esp_nom_m3day)
#num_stages = int(head_esp_nom_m / UniflocVBA.calc_ESP_head_m(q_esp_nom_m3day, pump_id = pump_id))
#for i in q_m3day:
#    h_m.append(UniflocVBA.calc_ESP_head_m(i,num_stages = num_stages,  pump_id = pump_id) / 100)
#    power_wt.append(UniflocVBA.calc_ESP_power_W(i, num_stages = num_stages, pump_id = pump_id) / 1000)
#    efficency_perc.append(UniflocVBA.calc_ESP_eff_fr(i, num_stages = num_stages, pump_id = pump_id) * 100)
#esp_curve = pd.DataFrame({'Напор, м/100': h_m, 'Мощность, кВт': power_wt, 'КПД, %.': efficency_perc})
#esp_curve.index = q_m3day
#plot_func(esp_curve, 'Характеристика ЭЦН')

In [ ]:
overall_data = overall_data[overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)'] != overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)']]

In [ ]:
overall_data['Относительная ошибка расчетов, %'] = ((overall_data['Q ж, м3/сут (Модель, вход) (ADAPT)'] - \
overall_data['Q ж, м3/сут (Модель, вход) (RESTORE)']) / overall_data['Q ж, м3/сут (Модель, вход) (ADAPT)']) * 100

only_error = overall_data[overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)'] != overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)']]
only_error = only_error['Относительная ошибка расчетов, %']

only_error = only_error.dropna()
only_error = only_error.abs()

only_error.mean()

In [ ]:
check_r2 = overall_data[['Q ж, м3/сут (Модель, вход) (ADAPT)','Q ж, м3/сут (Модель, вход) (RESTORE)']]

check_r2 = check_r2[check_r2['Q ж, м3/сут (Модель, вход) (ADAPT)'] != check_r2['Q ж, м3/сут (Модель, вход) (RESTORE)']]

check_r2 = check_r2.dropna()
check_r2.head()

In [ ]:
from sklearn.metrics import r2_score

coefficient_of_dermination = r2_score(check_r2['Q ж, м3/сут (Модель, вход) (ADAPT)'], check_r2['Q ж, м3/сут (Модель, вход) (RESTORE)'])

coefficient_of_dermination

In [ ]:
liquid_rates = ['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель, вход) (ADAPT)',
                  'Q ж, м3/сут (Модель, вход) (RESTORE)']

calibrs = ['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)',
               'Коэффициент калибровки по мощности - множитель (Модель, вход) (ADAPT)', 
                  'Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)',
               'Коэффициент калибровки по мощности - множитель (Модель, вход) (RESTORE)']

gor_wc = ['ГФ (модель, вход) (ADAPT)',
               'Обв, % (Модель, вход) (ADAPT)',
                  'ГФ (модель, вход) (RESTORE)',
               'Обв, % (Модель, вход) (RESTORE)']

temperatures = ['T устья, С (Модель, выход) (ADAPT)',
           'T прием, С (Модель, вход) (ADAPT)',
              'T устья, С (Модель, выход) (RESTORE)',
           'T прием, С (Модель, вход) (RESTORE)']

pressures = ['Рбуф (Ш) (ADAPT)',
            'P буф., атм (Модель, выход, адапт. Ш) (ADAPT)',
              'P прием ЭЦН, атм (Модель, вход) (ADAPT)',
              'P лин., атм (Модель, вход) (ADAPT)','Рлин ТМ (Ш) (ADAPT)',     
                       'P буф., атм (Модель, выход, адапт. Ш) (RESTORE)',
                   'P лин., атм (Модель, вход) (RESTORE)']

frequencies = ['F вращ ТМ (Ш) (ADAPT)',
               'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель, вход) (ADAPT)',
                       'F тока, ГЦ (Модель, вход) (RESTORE)']

beam_sizes = ['Dшт (Ш) (ADAPT)', 'Dшт (Ш) (RESTORE)']

powers = ['Активная мощность (СУ) (ADAPT)',
               'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (ADAPT)',
                       'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (RESTORE)']

currents= ['I, А (Модель, выход) (ADAPT)',
               'Ток фазы А (ADAPT)',
               'Ток фазы В (ADAPT)',
               'Ток фазы С (ADAPT)',
           'I, А (Модель, выход) (RESTORE)']

loads = ['Загрузка двигателя (СУ) (ADAPT)',
                           'Загрузка двигателя, (Модель, выход) (RESTORE)']

efficiency = ['КПД ЭЦН, д.ед. (Модель, выход) (ADAPT)',
                       'КПД ЭЦН, д.ед. (Модель, выход) (RESTORE)']

metrics = ['Значение функции ошибки (ADAPT)',
                  'Значение функции ошибки (RESTORE)',
          'Относительная ошибка расчетов, %']

dif_pressure = ['Перепад давления в ЭЦН, атм (ADAPT)',
                  'Перепад давления в ЭЦН, атм (RESTORE)']


In [ ]:
all_name_banches = [liquid_rates, 
                    calibrs, 
                    metrics, 
                    gor_wc, 
                    temperatures,
                    pressures, 
                    frequencies, 
                    beam_sizes,
                    powers, 
                    currents,
                    loads,
                    efficiency, 
                    dif_pressure]
subplot_titles = ['Дебиты', 
                  'Калибровки по напору и мощности', 
                  'Метрики расчета',
                  'ГФ, м3/м3 и Обводненность, %', 
                  'Температура, С', 
                  'Давления, атм', 
                  'Частота, Гц',
                  'Размер штуцера, мм', 
                  'Активная мощность, кВт', 
                  'Токи, А', 
                  'Загрузка, д.ед.', 
                  'КПД ЭЦН, д.ед.', 
                  'Перепад давления в ЭЦН, атм']

In [ ]:
def create_report_html(df, all_name_banches, subplot_titles, filename):
    subplot_amount = len(all_name_banches)
    fig = make_subplots(
    rows=subplot_amount, cols=1, shared_xaxes=True, 
    vertical_spacing= 0.01, 
    subplot_titles = subplot_titles

    )
    for i in range(subplot_amount):
        this_df = df[all_name_banches[i]]
        this_banch_trace = pw.create_traces_list_for_all_columms(this_df, chosen_mode='lines+markers')
        for j in this_banch_trace:
            fig.add_trace(j, row=i + 1, col=1)
        
    fig.layout.hovermode = 'x'
    fig.layout.height = 450 * subplot_amount
    plot(fig, filename  = filename + ' in one.html')

In [ ]:
create_report_html(overall_data, all_name_banches, subplot_titles, well_name)

In [ ]:
if one_restore == False:
    df = overall_data[['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель, вход) (ADAPT)',
                  'Q ж, м3/сут (Модель, вход) (RESTORE)',
                  'Q ж, м3/сут (Модель, вход) (RESTORE2)']]
else:
    df = overall_data[['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель, вход) (ADAPT)',
                  'Q ж, м3/сут (Модель, вход) (RESTORE)']]
plot_func(df, 'Дебиты')

In [ ]:
if one_restore == False:
    df = overall_data[['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)',
                   'Коэффициент калибровки по мощности - множитель (Модель, вход) (ADAPT)', 

                      'Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)',
                   'Коэффициент калибровки по мощности - множитель (Модель, вход) (RESTORE)', 

                       'Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE2)',
                   'Коэффициент калибровки по мощности - множитель (Модель, вход) (RESTORE2)'
                      ]]
else:
    df = overall_data[['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)',
               'Коэффициент калибровки по мощности - множитель (Модель, вход) (ADAPT)', 

                  'Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)',
               'Коэффициент калибровки по мощности - множитель (Модель, вход) (RESTORE)']]
plot_func(df, 'Калибровки')

In [ ]:
if one_restore == False:
    df = overall_data[['ГФ (модель, вход) (ADAPT)',
                   'Обв, % (Модель, вход) (ADAPT)',
                      'ГФ (модель, вход) (RESTORE)',
                   'Обв, % (Модель, вход) (RESTORE)',
                      'ГФ (модель, вход) (RESTORE2)',
                   'Обв, % (Модель, вход) (RESTORE2)']]
else:
    df = overall_data[['ГФ (модель, вход) (ADAPT)',
               'Обв, % (Модель, вход) (ADAPT)',
                  'ГФ (модель, вход) (RESTORE)',
               'Обв, % (Модель, вход) (RESTORE)']]
plot_func(df, 'ГФ и обводненность')

In [ ]:
if one_restore == False:
    df = overall_data[['T устья, С (Модель, выход) (ADAPT)',
               'T прием, С (Модель, вход) (ADAPT)',
                       'T устья, С (Модель, выход) (RESTORE)',
                       'T прием, С (Модель, вход) (RESTORE)',
                  'T устья, С (Модель, выход) (RESTORE2)',
               'T прием, С (Модель, вход) (RESTORE2)']]
else:
    df = overall_data[['T устья, С (Модель, выход) (ADAPT)',
           'T прием, С (Модель, вход) (ADAPT)',
              'T устья, С (Модель, выход) (RESTORE)',
           'T прием, С (Модель, вход) (RESTORE)']]
plot_func(df, 'Температура')

In [ ]:
if one_restore == False:
    df = overall_data[['Рбуф (Ш) (ADAPT)',
                       'P буф., атм (Модель, выход, адапт. Ш) (ADAPT)',
                   'P прием ЭЦН, атм (Модель, вход) (ADAPT)',
                   'P лин., атм (Модель, вход) (ADAPT)','Рлин ТМ (Ш) (ADAPT)',
                    
                       'P буф., атм (Модель, выход, адапт. Ш) (RESTORE)',
                   'P лин., атм (Модель, вход) (RESTORE)',
                       
                       'P буф., атм (Модель, выход, адапт. Ш) (RESTORE2)',
                   'P лин., атм (Модель, вход) (RESTORE2)'
                  ]]
    
else:
        df = overall_data[['Рбуф (Ш) (ADAPT)',
                           'P буф., атм (Модель, выход, адапт. Ш) (ADAPT)',
                   'P прием ЭЦН, атм (Модель, вход) (ADAPT)',
                   'P лин., атм (Модель, вход) (ADAPT)','Рлин ТМ (Ш) (ADAPT)',
                    
                       'P буф., атм (Модель, выход, адапт. Ш) (RESTORE)',
                   'P лин., атм (Модель, вход) (RESTORE)'
                  ]]
plot_func(df, 'Давления')

In [ ]:
if one_restore == False:
    df = overall_data[['F вращ ТМ (Ш) (ADAPT)',
               'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель, вход) (ADAPT)',
                   
                       'F тока, ГЦ (Модель, вход) (RESTORE)',
                   'F тока, ГЦ (Модель, вход) (RESTORE2)',
              ]]
else:
        df = overall_data[['F вращ ТМ (Ш) (ADAPT)',
               'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель, вход) (ADAPT)',
                   
                       'F тока, ГЦ (Модель, вход) (RESTORE)'
              ]]
plot_func(df, 'Частота тока')

In [ ]:
if one_restore == False:

    df = overall_data[['Dшт (Ш) (ADAPT)',
                       'Dшт (Ш) (RESTORE)',
                       'Dшт (Ш) (RESTORE2)'
                  ]]
else:
        df = overall_data[['Dшт (Ш) (ADAPT)',
                       'Dшт (Ш) (RESTORE)'
                  ]]
plot_func(df, 'Размер штуцера')

In [ ]:
if one_restore == False:
    df = overall_data[['Активная мощность (СУ) (ADAPT)',
               'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (ADAPT)',
                       'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (RESTORE)',
                   'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (RESTORE2)'
              ]]
else:
        df = overall_data[['Активная мощность (СУ) (ADAPT)',
               'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (ADAPT)',
                       'Акт. мощность на СУ, (Модель, выход, адапт. СУ) (RESTORE)'
                          ]]
#df['Активная мощность (СУ) (ADAPT)'] = df['Активная мощность (СУ) (ADAPT)'] * 1000
plot_func(df, 'Активная мощность')

In [ ]:
if one_restore == False:
    df = overall_data[['I, А (Модель, выход) (ADAPT)',
               'Ток фазы А (ADAPT)',
               'Ток фазы В (ADAPT)',
               'Ток фазы С (ADAPT)',
                   
                       'I, А (Модель, выход) (RESTORE)',
                       
                       'I, А (Модель, выход) (RESTORE2)'
              ]]
else:
        df = overall_data[['I, А (Модель, выход) (ADAPT)',
               'Ток фазы А (ADAPT)',
               'Ток фазы В (ADAPT)',
               'Ток фазы С (ADAPT)',
                                      'I, А (Модель, выход) (RESTORE)'
              ]]
plot_func(df, 'Токи')

In [ ]:
if one_restore == False:
    df = overall_data[['Загрузка двигателя (СУ) (ADAPT)',
               'Загрузка двигателя, (Модель, выход) (ADAPT)',
                       'Загрузка двигателя, (Модель, выход) (RESTORE)',
                       'Загрузка двигателя, (Модель, выход) (RESTORE2)'
              ]]
else:
        df = overall_data[['Загрузка двигателя (СУ) (ADAPT)',
                           'Загрузка двигателя, (Модель, выход) (RESTORE)'
              ]]
#df['Загрузка двигателя (СУ) (ADAPT)'] = df['Загрузка двигателя (СУ) (ADAPT)'] / 100
plot_func(df, 'Загрузка двигателя')

In [ ]:
if one_restore == False:
    df = overall_data[['КПД ЭЦН, д.ед. (Модель, выход) (ADAPT)',
                       'КПД ЭЦН, д.ед. (Модель, выход) (RESTORE)',
                  'КПД ЭЦН, д.ед. (Модель, выход) (RESTORE2)']]
else:
        df = overall_data[['КПД ЭЦН, д.ед. (Модель, выход) (ADAPT)',
                       'КПД ЭЦН, д.ед. (Модель, выход) (RESTORE)']]
    
plot_func(df, 'КПД ЭЦН')

In [ ]:
if one_restore == False:
    df = overall_data[['Значение функции ошибки (ADAPT)',
                  'Значение функции ошибки (RESTORE)',
                  'Значение функции ошибки (RESTORE2)']]
else:
        df = overall_data[['Значение функции ошибки (ADAPT)',
                  'Значение функции ошибки (RESTORE)']]
    
plot_func(df, 'Значение функции ошибки (ADAPT)')

In [ ]:
if one_restore == False:
    df = overall_data[['Перепад давления в ЭЦН, атм (ADAPT)',
                  'Перепад давления в ЭЦН, атм (RESTORE)',
                  'Перепад давления в ЭЦН, атм (RESTORE2)']]
else:
        df = overall_data[['Перепад давления в ЭЦН, атм (ADAPT)',
                  'Перепад давления в ЭЦН, атм (RESTORE)'
                          ]]
    
plot_func(df, 'Перепад давления в ЭЦН, атм')